In [2]:
entities = [{'name': 'Shane Kimbrough', 'id': 910512, 'db_id': 0, 'db_name': None, 'classification_score': 0.9988277554512024, 'classification_label': 0, 'annotation_label': None, 'found_name': 'Shane Kimbrough', 'found_position': 818}, {'name': 'Donald Trump', 'id': 910513, 'db_id': 9241, 'db_name': 'Donald Trump', 'classification_score': 0.998790442943573, 'classification_label': 0, 'annotation_label': None, 'found_name': 'Donald Trump', 'found_position': 1058}, {'name': 'Joe Biden', 'id': 910514, 'db_id': 0, 'db_name': None, 'classification_score': 0.9988563656806946, 'classification_label': 0, 'annotation_label': None, 'found_name': 'Joe Biden', 'found_position': 1092}, {'name': 'Kamala Harris', 'id': 910515, 'db_id': 0, 'db_name': None, 'classification_score': 0.9985531568527222, 'classification_label': 0, 'annotation_label': None, 'found_name': 'Kamala Harris', 'found_position': 1340}, {'name': 'Hillary Clintoné', 'id': 910516, 'db_id': 0, 'db_name': None, 'classification_score': 0.9980640411376953, 'classification_label': 0, 'annotation_label': None, 'found_name': 'Hillary Clintoné', 'found_position': 1529}, {'name': 'Trump', 'id': 910517, 'db_id': 9241, 'db_name': 'Donald Trump', 'classification_score': 0.9979464411735535, 'classification_label': 0, 'annotation_label': None, 'found_name': 'Trump', 'found_position': 1791}, {'name': 'Trump', 'id': 910518, 'db_id': 9241, 'db_name': 'Donald Trump', 'classification_score': 0.9976881742477417, 'classification_label': 0, 'annotation_label': None, 'found_name': 'Trumpról', 'found_position': 2023}, {'name': 'Trump', 'id': 910519, 'db_id': 9241, 'db_name': 'Donald Trump', 'classification_score': 0.998136043548584, 'classification_label': 0, 'annotation_label': None, 'found_name': 'Trump', 'found_position': 2159}, {'name': 'Trump', 'id': 910520, 'db_id': 9241, 'db_name': 'Donald Trump', 'classification_score': 0.9907723665237427, 'classification_label': 0, 'annotation_label': None, 'found_name': 'Trump', 'found_position': 2330}, {'name': 'Kate Rubins', 'id': 910521, 'db_id': 0, 'db_name': None, 'classification_score': 0.9979288578033447, 'classification_label': 0, 'annotation_label': None, 'found_name': 'Kate Rubins', 'found_position': 2528}]


In [22]:
entities

[{'name': 'Shane Kimbrough',
  'id': 910512,
  'db_id': 0,
  'db_name': None,
  'classification_score': 0.9988277554512024,
  'classification_label': 0,
  'annotation_label': None,
  'found_name': 'Shane Kimbrough',
  'found_position': 818},
 {'name': 'Donald Trump',
  'id': 910513,
  'db_id': 9241,
  'db_name': 'Donald Trump',
  'classification_score': 0.998790442943573,
  'classification_label': 0,
  'annotation_label': None,
  'found_name': 'Donald Trump',
  'found_position': 1058},
 {'name': 'Joe Biden',
  'id': 910514,
  'db_id': 0,
  'db_name': None,
  'classification_score': 0.9988563656806946,
  'classification_label': 0,
  'annotation_label': None,
  'found_name': 'Joe Biden',
  'found_position': 1092},
 {'name': 'Kamala Harris',
  'id': 910515,
  'db_id': 0,
  'db_name': None,
  'classification_score': 0.9985531568527222,
  'classification_label': 0,
  'annotation_label': None,
  'found_name': 'Kamala Harris',
  'found_position': 1340},
 {'name': 'Hillary Clintoné',
  'id': 9

In [35]:
from typing import Literal, Optional
from collections import defaultdict


def group_dicts_by_name(dict_list):
    groups = []
    visited = set()  # To keep track of the processed dictionaries

    for i, dict_i in enumerate(dict_list):
        if i in visited:
            continue  # Skip if this dictionary has already been grouped

        current_group = [dict_i]
        visited.add(i)

        for j, dict_j in enumerate(dict_list):
            if j in visited:
                continue

            # Check if either name is a substring of the other
            if dict_i["name"] in dict_j["name"] or dict_j["name"] in dict_i["name"]:
                current_group.append(dict_j)
                visited.add(j)

        groups.append(current_group)

    return groups


def map_entities(entities: list[dict]):
    def normalized_score(e):
        label: Literal[1] | Literal[-1] = 1 if e["classification_label"] else -1
        score: float = e["classification_score"]
        return (score * label + 1) / 2

    def avg(l):
        return sum(l) / len(l) if l else 0

    entity_map: dict[Optional[int], list] = defaultdict(list)

    for entity in entities:
        entity_map[entity["db_id"]].append(entity)

    map_non_db = group_dicts_by_name(entity_map[0])
    entity_list = []

    for db_id, entity_group in entity_map.items():
        if db_id is not None and db_id != 0:
            score = avg([normalized_score(e) for e in entity_group])
            label = 1 if score > 0.5 else 0
            new_label = 1 if score > 0.5 else -1
            old_score = (score / 2 - 1) * new_label
            entity = {
                "db_id": db_id,
                "name": entity_group[0]["db_name"],
                "score": score,
                "classification_score": old_score,
                "classification_label": label,
                "occurences": entity_group,
            }
            entity_list.append(entity)
    for entity_group in map_non_db:
        score = avg([normalized_score(e) for e in entity_group])
        label = 1 if score > 0.5 else 0
        new_label = 1 if score > 0.5 else -1
        old_score = (score / 2 - 1) * new_label
        entity = {
            "db_id": entity_group[0]["db_id"],
            "name": entity_group[0]["name"],
            "score": score,
            "classification_score": old_score,
            "classification_label": label,
            "occurences": entity_group,
        }
        entity_list.append(entity)

    return entity_list


map_entities(entities)

[{'db_id': 9241,
  'name': 'Donald Trump',
  'score': 0.0016666531562805175,
  'classification_score': 0.9991666734218597,
  'classification_label': 0,
  'occurences': [{'name': 'Donald Trump',
    'id': 910513,
    'db_id': 9241,
    'db_name': 'Donald Trump',
    'classification_score': 0.998790442943573,
    'classification_label': 0,
    'annotation_label': None,
    'found_name': 'Donald Trump',
    'found_position': 1058},
   {'name': 'Trump',
    'id': 910517,
    'db_id': 9241,
    'db_name': 'Donald Trump',
    'classification_score': 0.9979464411735535,
    'classification_label': 0,
    'annotation_label': None,
    'found_name': 'Trump',
    'found_position': 1791},
   {'name': 'Trump',
    'id': 910518,
    'db_id': 9241,
    'db_name': 'Donald Trump',
    'classification_score': 0.9976881742477417,
    'classification_label': 0,
    'annotation_label': None,
    'found_name': 'Trumpról',
    'found_position': 2023},
   {'name': 'Trump',
    'id': 910519,
    'db_id': 9241